# Text Generation

## Introduction

Markov chains can be used for very basic text generation. Think about every word in a corpus as a state. We can make a simple assumption that the next word is only dependent on the previous word - which is the basic assumption of a Markov chain.

Markov chains don't generate text as well as deep learning, but it's a good (and fun!) start.

## Select Text to Imitate

In this notebook, we're specifically going to generate text in the style of Ali Wong, so as a first step, let's extract the text from her comedy routine.

In [31]:
# Read in the corpus, including punctuation!
import pandas as pd

data = pd.read_pickle('corpus2.pkl')
data.head()

,transcript,name
file1,what a wooden structure says about our ancien...,file1
file10,how fiefdoms and do or die imperil the success...,file10
file11,years later and still no clarity on green cle...,file11
file12,a newfound neutron star might light the way fo...,file12
file13,uttering the uterus mapping myths and menstrua...,file13


In [32]:
# Extract only one file text
file = data.transcript.loc['file41']
file[:200]

'the suns activity cycle is reaching its peak early what it meansphotodavid matosonunsplashnew delhias the world marks world brain stroke day on sunday a lack of optimal data from the southeast asia re'

## Build a Markov Chain Function

We are going to build a simple Markov chain function that creates a dictionary:
* The keys should be all of the words in the corpus
* The values should be a list of the words that follow the keys

In [33]:
from collections import defaultdict

def markov_chain(text):
    '''The input is a string of text and the output will be a dictionary with each word as
       a key and each value as the list of words that come after the key in the text.'''
    
    # Tokenize the text by word, though including punctuation
    words = text.split(' ')
    
    # Initialize a default dictionary to hold all of the words and next words
    m_dict = defaultdict(list)
    
    # Create a zipped list of all of the word pairs and put them in word: list of next words format
    for current_word, next_word in zip(words[0:-1], words[1:]):
        m_dict[current_word].append(next_word)

    # Convert the default dict back into a dictionary
    m_dict = dict(m_dict)
    return m_dict

In [34]:
# Create the dictionary for data routine, take a look at it
data_dict = markov_chain(file)
data_dict

{'the': ['suns',
  'world',
  'southeast',
  'fight',
  'burden',
  '',
  'global',
  'paper',
  'brain',
  'artery',
  'artery',
  'paper',
  'proportion',
  'population',
  'other',
  'new',
  'case',
  'time',
  'lack',
  'region',
  'paper',
  'huge',
  'prevalence',
  'fact',
  'country',
  'lead',
  'neurology',
  'former',
  'latter',
  'case',
  'case',
  'data',
  'family',
  'line',
  'data',
  'northern',
  'entire',
  'highest',
  'overall',
  'sear',
  'highest',
  'morbidity',
  'country',
  'dalys',
  'disease',
  'gbd',
  'authors',
  'highest',
  'paper',
  'largest',
  'economy',
  'country',
  'reasons',
  'developed',
  'small',
  'paper',
  'bloodstream',
  'arteries',
  'brain',
  'majority',
  'northeastern',
  'outliers',
  'possible',
  'habit',
  'women',
  'dehydration',
  'cmc',
  'paper',
  'sear',
  'global',
  'sear',
  'paper',
  'contributing',
  'paper',
  'specific',
  'body',
  'brain',
  'inflammatory',
  'dengue',
  'common',
  'body',
  'ischemic'

In [57]:
def apply_markov_chain(row):
    result = markov_chain(row['transcript'])
    return result

# Apply the function to each row
data_dict = data.apply(apply_markov_chain, axis=1)
data_dict.head()

file1     {'what': ['a', 'it'], 'a': ['', 'vision', 'bid...
file10    {'how': ['fiefdoms'], 'fiefdoms': ['and'], 'an...
file11    {'': ['years', 'students', 'of', 'of', 'minute...
file12    {'a': ['newfound', 'new', 'classroom', 'school...
file13    {'uttering': ['the'], 'the': ['uterus', 'way',...
dtype: object

## Create a Text Generator

We're going to create a function that generates sentences. It will take two things as inputs:
* The dictionary you just created
* The number of words you want generated

Here are some examples of generated sentences:

>'Bangladesh india according to people would have come to strokes can be one of a.'

>'Period of the family members of  also lead authors said it to monitor stroke.'

In [62]:
import random

def generate_sentence(chain, count=15):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the number of words you would like to see in your generated sentence.'''

    # Capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = word1.capitalize()

    # Generate the second word from the value list. Set the new word as the first word. Repeat.
    for i in range(count-1):
        if word1 not in chain:
            break 
        word2 = random.choice(chain[word1])
        word1 = word2
        sentence += ' ' + word2

    # End it with a period
    sentence += '.'
    return(sentence)

In [63]:
generate_sentence(data_dict['file4'])

'Candidates is common labs remain open throughout the semester fee hikes dissolve emergency fundsrecently the.'

In [64]:
def apply_generate_sentence(row):
    result = generate_sentence(row)
    return result

# Apply the function to each row
data_sentence = data_dict.apply(apply_generate_sentence)

In [65]:
data_sentence

file1     Looking to have more detail belowalso readthe ...
file10    Hierarchical classification of keeping the hie...
file11    Huge numbers of education should be viewed as ...
file12    In a phd in particular for brave schoolshere a...
file13    Mess it is the introduction tospace segregatio...
file14    Syntax errors google docs finishes sentences a...
file15    Conservative and really ison the paperud what ...
file16    Sahara or north africa this highlights for dri...
file17    Molecule found in red blood of these treatment...
file18    While chatgpt and mathematical concepts what c...
file19    Domesticated crops – such limited traits of ne...
file2     Sriram ganeshan ccny faculty and stage directo...
file20    Attractive mates many sites in southwestern fr...
file21    Filtration or its severitywhen looking at grea...
file22    Fragile and what extent is easier to scientist...
file23    Sites with the southern jordan is unclearearly...
file24    Holding the containers hollow 

### Assignment:
1. Generate sentence for other files also.
2. Try making the generate_sentence function better. Maybe allow it to end with a random punctuation mark or end whenever it gets to a word that already ends with a punctuation mark.

In [ ]:
import random
import string

def generate_sentence(chain, max_length=15):
    '''Input a dictionary in the format of key = current word, value = list of next words
       along with the maximum number of words you would like to see in your generated sentence.'''

    # Capitalize the first word
    word1 = random.choice(list(chain.keys()))
    sentence = [word1.capitalize()]

    # Generate subsequent words
    for _ in range(max_length - 1):
        word2 = random.choice(chain.get(word1, ['']))
        if not word2:  # If no word is available, end the sentence
            break
        if word2.endswith(('.', '!', '?')):  # End sentence if the word ends with a punctuation mark
            sentence.append(word2)
            break
        sentence.append(word2)
        word1 = word2

    # End the sentence with a random punctuation mark if not already ended
    if not sentence[-1].endswith(('.', '!', '?')):
        sentence[-1] += random.choice(string.punctuation)

    return ' '.join(sentence)

In [74]:
generate_sentence(data_dict['file10'],25)

'Publications on each topic and dropped out of language obstaclesmedical terminologies are we require a veryradical idea adapt the background have deep meaningsbeing from aiims.'

In [75]:
generate_sentence(data_dict['file31'],20)

'Factors ofsleep hygiene daily activities including music therapy behavioural state exhibited by tech billionaire narayana murthy whomade a discrepancy was.'